In [1]:
import pandas as pd


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
import graphviz, pydot, pydotplus
from sklearn.model_selection import train_test_split


In [3]:
root_folder = "h"
healthy = []
for root, dirs, files in os.walk(root_folder):  
    for filename in files:
        healthy.append(filename)
print(healthy)

['image_cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900 (2).txt', 'image_cortex_patient20022019_633nm_obj20_power100_1sec_сenter2900 (1).txt', 'image_cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900 (1).txt', 'image_cortex_patient20022019_633nm_obj20_power100_1sec_сenter2900.txt', 'image_cortex_patient13012020_633nm_obj20_power100_1sec_step4um_сenter2900.txt', 'image_cortex_patient25022020_633nm_obj20_power100_1sec_сenter2900.txt', 'image_cortex_patient15012020_633nm_obj20_power100_2sec_acc1_сenter2900.txt', 'image_cortex_patient78_633nm_obj20_power100_step4um_2sec_acc1_сenter2900.txt', 'image_cortex_patient25022020_633nm_obj20_power100_2sec_сenter2900.txt', 'image_cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900.txt']


In [4]:
measurements = []
classifications = []
points = []

In [5]:
count_healthy = 0

for name in healthy:
    df = pd.read_csv("h/" + name, sep='\t',skiprows=[0],
                     header=None, names=['X', 'Y', 'Wave', 'Intensity'])
    for i in range(len(df)//1015):
        measurements.append(df[['Intensity']][i*1015:(i+1)*1015].to_numpy())
        classifications.append([1,0])
        points.append(df[["X", "Y"]].iloc[i * 1015].values)
        count_healthy += 1

In [6]:
root_folder = "s/"
sick = []
for root, dirs, files in os.walk(root_folder):  
    for filename in files:
        sick.append(filename)

In [7]:
count_sick = 0

for name in sick:
    df = pd.read_csv("s/" + name, sep='\t',skiprows=[0],
                     header=None, names=['X', 'Y', 'Wave', 'Intensity'])
    for i in range(len(df)//1015):
        measurements.append(df[['Intensity']][i*1015:(i+1)*1015].to_numpy())
        classifications.append([0,1])
        count_sick += 1

In [8]:
X = np.asarray(measurements)
y = np.asarray(classifications)

In [9]:
X.shape

(6348, 1015, 1)

In [10]:
X = X.reshape(-1, 1015)
X.shape

(6348, 1015)

In [11]:
y.shape

(6348, 2)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f'Total amount of train measurements: {X_train.shape}')
print(f'Total amount of train labels: {y_train.shape}')
print(f'Total amount of test measurements: {X_test.shape}')
print(f'Total amount of test labels: {y_test.shape}')
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

Total amount of train measurements: (4761, 1015)
Total amount of train labels: (4761, 2)
Total amount of test measurements: (1587, 1015)
Total amount of test labels: (1587, 2)


In [13]:
#from scipy.signal import savgol_filter
#for i in range(len(X_train)):
 #   X_train[i] = savgol_filter(X_train[i], 101, 2)
#for i in range(len(X_test)):
 #   X_test[i] = savgol_filter(X_test[i], 101, 2)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from optuna.samplers import TPESampler
def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 2000, log=True),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "max_depth": trial.suggest_int("max_depth", 5, 50, log=True),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 200, log=True),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 200, log=True),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [False, True]),
    }
    rf_cls = RandomForestClassifier(**param)

    rf_cls.fit(X_train, y_train_labels)

    preds = rf_cls.predict(X_test)
    f1 = f1_score(y_test_labels, preds)
    acc = accuracy_score(y_test_labels, preds)
    print(f"{f1}, {acc}")
    return f1

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import optuna

sampler = TPESampler(seed=2)
study = optuna.create_study(study_name="rf", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100000)

[I 2023-05-15 00:42:54,107] A new study created in memory with name: rf
[I 2023-05-15 00:43:31,953] Trial 0 finished with value: 0.831479897348161 and parameters: {'n_estimators': 248, 'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.831479897348161.


0.831479897348161, 0.8758664146187776


[I 2023-05-15 00:43:34,477] Trial 1 finished with value: 0.7051397655545537 and parameters: {'n_estimators': 81, 'criterion': 'log_loss', 'max_depth': 35, 'min_samples_split': 17, 'min_samples_leaf': 80, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.831479897348161.


0.7051397655545537, 0.7939508506616257


[I 2023-05-15 00:43:36,584] Trial 2 finished with value: 0.7659574468085105 and parameters: {'n_estimators': 79, 'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.831479897348161.


...

0.8713910761154856, 0.9073724007561437


[I 2023-05-15 10:21:15,282] Trial 598 finished with value: 0.8460861917326298 and parameters: {'n_estimators': 59, 'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 19, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 223 with value: 0.8806228373702423.


0.8460861917326298, 0.8897290485192186


[W 2023-05-15 10:21:39,526] Trial 599 failed with parameters: {'n_estimators': 103, 'criterion': 'entropy', 'max_depth': 22, 'min_samples_split': 14, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/ym/m5symfp11nzfwgr30yqmsw3xtkhq7t/T/ipykernel_59328/2241224304.py", line 16, in objective
    rf_cls.fit(X_train, y_train_labels)
  File "/opt/homebrew/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 476, in fit
    trees = Parallel(
  File "/opt/homebrew/lib/python3.10/site-packages/joblib/parallel.py", line 1088, in __call__
    while self.dispatch_one_batch(iterator):
  File "/opt/homebrew/lib/python3.10/site-packages/joblib/parallel.py", line 901, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/homebr

KeyboardInterrupt: 

In [16]:
bp = study.best_params
rf_cls = RandomForestClassifier(**bp)

rf_cls.fit(X_train, y_train_labels)

preds = rf_cls.predict(X_test)


In [17]:
print(bp)

{'n_estimators': 119, 'criterion': 'entropy', 'max_depth': 34, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}


In [19]:
print(f'Random forest train F1 Score {f1_score(y_train_labels, rf_cls.predict(X_train))}')
print(f'Random forest train Accuracy {accuracy_score(y_train_labels, rf_cls.predict(X_train))}')
print(f'Random forest test  F1 Score {f1_score(y_test_labels, rf_cls.predict(X_test))}')
print(f'Random forest test  Accuracy {accuracy_score(y_test_labels, rf_cls.predict(X_test))}')

Random forest train F1 Score 1.0
Random forest train Accuracy 1.0
Random forest test  F1 Score 0.8698689956331879
Random forest test  Accuracy 0.906112161310649


In [22]:
import pickle
with open('rf-clf.pkl', 'wb') as fout:
    pickle.dump(rf_cls, fout)